# Project 3: Web APIs and NLP <br>

#### Brandie Hatch

## Problem



Learn with Chewie presents:
Web API and NLP Services

## Preparation

In [49]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

%matplotlib inline
pd.options.display.max_columns =999

import requests
import time
import re
import nltk

In [60]:
# Gather Data
import praw

reddit = praw.Reddit(
    client_id = '5aTMVLqpXQvo6vpgNv171w',
    client_secret = 'FAjO19Zyx5W4N9llEeklP8apwz20Zw',
    username='BrandieHatch',
    password='Qwert123',
    user_agent='praw_BH',
)
print(reddit.read_only)

False


In [51]:
subreddit = reddit.subreddit('Dogtraining')

print(subreddit.display_name)
print(subreddit.title)
print(subreddit.description)

Dogtraining
DogTraining: For training and rehabilitating dogs and puppies and all things canine behavior
###Welcome
This is a forum on dog training and behavior that focuses on a [least intrusive, minimally aversive approach.](https://iaabc.org/about/LIMA)    
   
The advice here is not a replacement for professional help. If your training is not fun and effective, or if you need additional help, then please [find a certified trainer](http://www.reddit.com/r/Dogtraining/wiki/findingatrainer) for assistance.  
  
----   
  
###Filters   
[All | Remove Filter](http://reddit.com/r/Dogtraining/)   
[Academic](http://ac.reddit.com/r/dogtraining/search?q=flair%3Aacademic&restrict_sr=on&sort=new&t=all#ac) [Announcement](http://an.reddit.com/r/Dogtraining/search?q=flair%3Aannouncing&restrict_sr=on&sort=new&t=all#an)  
[Brags](http://br.reddit.com/r/Dogtraining/search?q=flair%3Abrags&restrict_sr=on&sort=new&t=all#br) [Community](http://cm.reddit.com/r/Dogtraining/search?q=flair%3Acommunity&rest

In [52]:
hot_dogtraining = subreddit.hot(limit=1)

for submission in hot_dogtraining:
    print(dir(submission))   # can see what all is connected

['STR_FIELD', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_chunk', '_comments_by_id', '_fetch', '_fetch_data', '_fetch_info', '_fetched', '_kind', '_reddit', '_reset_attributes', '_safely_add_arguments', '_url_parts', '_vote', 'all_awardings', 'allow_live_comments', 'approved_at_utc', 'approved_by', 'archived', 'author', 'author_flair_background_color', 'author_flair_css_class', 'author_flair_richtext', 'author_flair_template_id', 'author_flair_text', 'author_flair_text_color', 'author_flair_type', 'author_fullname', 'author_is_blocked', 'author_patreon_flair', 'author_premium', 'award', 'awarders', 'banned_at_utc', 'banned_by', 'can_gild', 'can_mod_post', 'category', 'clear_

In [53]:
hot_dogtraining = subreddit.hot(limit=5)

for submission in hot_dogtraining:
    print(submission.ups)   
    print(submission.downs)
    print(submission.title)

8
0
Trick of the Month - May 2022 - Crawl Backwards
7
0
Announcement - Puppy Enrichment AMA With Allie Bender! Sunday 5/22 1PM EST
344
0
10 Month old cocker spaniel agression , he's very excited for walks , but it's very hard to wear and remove the harness and he ends up like this. Please help!
5
0
How do I get a cafe/brewery dog?
4
0
Dog suddenly peeing?


In [58]:
hot_dogtraining = subreddit.hot(limit=5)

for submission in hot_dogtraining:
    if not submission.stickied:
        print('Title: {}, ups: {}, downs: {}'.format(submission.title, submission.ups, submission.downs))

Title: 10 Month old cocker spaniel agression , he's very excited for walks , but it's very hard to wear and remove the harness and he ends up like this. Please help!, ups: 341, downs: 0
Title: How do I get a cafe/brewery dog?, ups: 5, downs: 0
Title: Dog suddenly peeing?, ups: 4, downs: 0


In [67]:
subreddit_dogs = reddit.subreddit('dogtraining+dogcare').hot(limit=5)

#subs = []

for submission in subreddit_dogs:
    if not submission.stickied:
        subs = {'subreddit: {}, id: {}, Title: {}, author: {}, ups: {}, downs: {}, selftext: {}, score: {}'.format(submission.subreddit, submission.id, submission.title, submission.author, submission.ups, submission.downs, submission.selftext, submission.score)}
        post_df = pd.DataFrame(data=subs)



In [78]:
#def posts(subreddit):

#subs_dict = {}

subreddit_dogs = reddit.subreddit('dogtraining+dogcare').hot(limit=5)

for submission in subreddit_dogs:
    if not submission.stickied:

        df = pd.DataFrame([['subreddit: {}, id: {}, Title: {}, author: {}, ups: {}, downs: {}, selftext: {}, score: {}'.format(submission.subreddit, submission.id, submission.title, submission.author, submission.ups, submission.downs, submission.selftext, submission.score)]])


#def posts_to_df(posts):
    #data = posts['data']
    
    #df = pd.DataFrame(data)
    #return df[['subreddit', 'id', 'author', 'score', 'title', 'selftext', 'num_comments', 'created_utc']]




In [79]:
df.head()

,0
0,"subreddit: Dogtraining, id: uoc80s, Title: “Wa..."


In [68]:
post_df.head()

,0
0,"subreddit: Dogtraining, id: uoc80s, Title: “Wa..."


In [13]:
response = requests.get(
    'http://reddit.com/r/Dogtraining/',
    params={'q': 'requests+language:python'},
)

In [14]:
requests.Response()

<Response [None]>

In [ ]:
url = 'http://reddit.com/r/Dogtraining/'
params = {}
x = requests.get(url, params)
print(x.json())

In [33]:
def get_posts_batch(subreddit, before):
    
    url = 'https://api.pushshift.io/reddit/search/submission'
    
    params = {
        'subreddit': subreddit,
        'size': 100
    }
    if before is not None: 
        params['before'] = before
        
    res = requests.get(url, params)

    return res.json()

def posts_to_df(posts):
    data = posts['data']
    
    df = pd.DataFrame(data)
    return df[['subreddit', 'id', 'author', 'score', 'title', 'selftext', 'num_comments', 'created_utc']]

def get_posts(subreddit, num_posts):
    
    post_ids = set()
    
    before = None
    
    i = 0
    
    while len(post_ids) < num_posts:
        batch = get_posts_batch(subreddit, before)
        
        df = posts_to_df(batch)
        
        before = df['created_utc'].min()
        
        post_ids.update(df['id'])
        
        if i == 0: 
            header = True
            mode = 'w'
        else: 
            header = False 
            mode = 'a'
        df.to_csv(subreddit + '.csv', mode=mode, header=header, index=False)
        print(f'We have scraped {len(post_ids)} posts.')
        i += 1
        time.sleep(2)

In [37]:
get_posts('dogtraining', 5000)

We have scraped 100 posts.
We have scraped 200 posts.
We have scraped 300 posts.
We have scraped 400 posts.
We have scraped 500 posts.
We have scraped 600 posts.
We have scraped 700 posts.
We have scraped 800 posts.
We have scraped 900 posts.
We have scraped 1000 posts.
We have scraped 1100 posts.
We have scraped 1200 posts.
We have scraped 1300 posts.
We have scraped 1400 posts.
We have scraped 1500 posts.
We have scraped 1600 posts.
We have scraped 1700 posts.
We have scraped 1800 posts.
We have scraped 1900 posts.
We have scraped 2000 posts.
We have scraped 2100 posts.
We have scraped 2200 posts.
We have scraped 2300 posts.
We have scraped 2400 posts.
We have scraped 2500 posts.
We have scraped 2600 posts.
We have scraped 2700 posts.
We have scraped 2800 posts.
We have scraped 2900 posts.
We have scraped 3000 posts.
We have scraped 3100 posts.
We have scraped 3199 posts.
We have scraped 3299 posts.
We have scraped 3399 posts.
We have scraped 3499 posts.
We have scraped 3599 posts.
W

In [39]:
dogtraining = pd.read_csv('./dogtraining.csv')
dogtraining.head()

,subreddit,id,author,score,title,selftext,num_comments,created_utc
0,Dogtraining,up55oy,devitosecret,1,Introducing big puppy to smaller older dog?,I have a 5.5 month old GSD who gets very excit...,0,1652487326
1,Dogtraining,up52vw,slothsandwhich,1,How do I get a cafe/brewery dog?,I am sitting at a brewery right now and all th...,0,1652487073
2,Dogtraining,up4qcr,L1Llunch,1,Not paying attention to me outside of my bedroom.,"i have a year old german shepherd, have only h...",2,1652485953
3,Dogtraining,up4b6t,Affectionate-Box-276,1,6 month old husky/ shepherd mix suddenly terri...,We adopted a husky/ shepherd mix about 3 weeks...,1,1652484573
4,Dogtraining,up49sh,Live_Damage_2902,1,why does my Dog get excited when certain peopl...,I've noticed that when my phone rings it gets ...,0,1652484449


In [40]:
dogtraining.isnull().sum()

subreddit         0
id                0
author            0
score             0
title             0
selftext        596
num_comments      0
created_utc       0
dtype: int64

In [41]:
get_posts('dogcare', 5000)

We have scraped 100 posts.
We have scraped 200 posts.
We have scraped 300 posts.
We have scraped 400 posts.
We have scraped 500 posts.
We have scraped 600 posts.
We have scraped 700 posts.
We have scraped 800 posts.
We have scraped 900 posts.
We have scraped 1000 posts.
We have scraped 1100 posts.
We have scraped 1200 posts.
We have scraped 1300 posts.
We have scraped 1400 posts.
We have scraped 1500 posts.
We have scraped 1600 posts.
We have scraped 1700 posts.
We have scraped 1799 posts.
We have scraped 1899 posts.
We have scraped 1999 posts.
We have scraped 2099 posts.
We have scraped 2199 posts.
We have scraped 2299 posts.
We have scraped 2399 posts.
We have scraped 2499 posts.
We have scraped 2599 posts.
We have scraped 2699 posts.
We have scraped 2799 posts.
We have scraped 2899 posts.
We have scraped 2999 posts.
We have scraped 3099 posts.
We have scraped 3199 posts.
We have scraped 3299 posts.
We have scraped 3399 posts.
We have scraped 3499 posts.
We have scraped 3599 posts.
W

In [43]:
dogcare = pd.read_csv('./dogcare.csv')
dogcare.head()

,subreddit,id,author,score,title,selftext,num_comments,created_utc
0,DogCare,uoyffw,Disastrous_Bobcat402,1,Anyone know what this could be? 7 Yr old Irish...,NaN,0,1652467478
1,DogCare,uoxknd,PerformanceStraight,1,Gum stuck in paw,[removed],0,1652465017
2,DogCare,uotky9,Routine_Heart2153,1,Nala's Crucial ACL Surgery,[removed],0,1652453786
3,DogCare,uokuoz,PrincessStorm97,1,Enrichment that doesn’t involve treats or exer...,Sad to say my oldest has heart worms. We start...,0,1652420750
4,DogCare,uohszh,LaTesora,1,New Husky/German Shepherd owner looking for ad...,Hi y'all!\n\nI just adopted a 5 year old Germa...,0,1652409660


In [48]:
dogcare.isnull().sum()

subreddit          0
id                 0
author             0
score              0
title              0
selftext        2180
num_comments       0
created_utc        0
dtype: int64

## Explore Data

In [2]:
# Imports



## Model

## Model Evaluation

## Answer Problem, Conclusions, and Recommendations

## Sources